In [1]:
!pip install groq

   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   --------------------- ------------------ 1.0/2.0 MB 5.0 MB/s eta 0:00:01
   ---------------------------------------- 2.0/2.0 MB 4.9 MB/s eta 0:00:00

   --- ------------------------------------  1/13 [sniffio]
   ------ ---------------------------------  2/13 [pydantic-core]
   --------- ------------------------------  3/13 [idna]
   --------- ------------------------------  3/13 [idna]
   ------------ ---------------------------  4/13 [h11]
   --------------- ------------------------  5/13 [distro]
   ------------------ ---------------------  6/13 [certifi]
   ------------------------ ---------------  8/13 [pydantic]
   ------------------------ ---------------  8/13 [pydantic]
   ------------------------ ---------------  8/13 [pydantic]
   ------------------------ ---------------  8/13 [pydantic]
   ------------------------ ---------------  8/13 [pydantic]
   ------------------------ ---------------  8/13 [pydanti

In [ ]:
from groq import Groq

client = Groq(api_key="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")

def ask_llama3(question, max_tokens=100):  # Default: 100 tokens
    response = client.chat.completions.create(
        model="llama3-70b-8192",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},  # Encourage brevity
            {"role": "user", "content": question}
        ],
        temperature=0.7,
        max_tokens=max_tokens,
    )
    return response.choices[0].message.content

# Test with token limit
question = "How many days should a Mastech Infotrellis employee come to office"
answer = ask_llama3(question, max_tokens=400) 
print("Llama 3 70B Response:\n", answer)

Llama 3 70B Response:
 Mastech Infotrellis's company policy!

According to Mastech Infotrellis's official sources, the company follows a flexible work-from-home policy. This means that employees are not required to come to the office for a fixed number of days. Instead, they can work remotely and manage their work schedules as per their convenience.

However, it's possible that team leads or managers might schedule in-person meetings, training sessions, or collaboration activities that may require employees to come to the office occasionally.

To confirm the exact policy, I recommend checking with your HR department or supervisor for the most up-to-date and specific information on work-from-home arrangements.

Remember to prioritize your productivity, comfort, and work-life balance while working from home or in the office!


In [13]:
!pip install langchain sentence-transformers chromadb langchain-community

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/10.4 MB ? eta -:--:--
   -- ------------------------------------- 0.5/10.4 MB 4.2 MB/s eta 0:00:03
   ---- ----------------------------------- 1.0/10.4 MB 4.6 MB/s eta 0:00:03
   ----- ---------------------------------- 1.3/10.4 MB 3.0 MB/s eta 0:00:03
   -------- ------------------------------- 2.1/10.4 MB 2.4 MB/s eta 0:00:04
   ----------- ---------------------------- 2.9/10.4 MB 2.9 MB/s eta 0:00:03
   ------------ --------------------------- 3.1/10.4 MB 2.7 MB/s eta 0:00:03
   ------------ --------------------------- 3.1/10.4 MB 2.7 MB/s eta 0:00:03
   ------------ --------------------------- 3.1/10.4 MB 2.7 M

In [28]:
from groq import Groq
from langchain_community.document_loaders import JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma

# Initialize Groq client
client = Groq(api_key="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")

In [29]:
def load_json(file_path):
    loader = JSONLoader(
        file_path=file_path,
        jq_schema=".[] | {question: .question, answer: .answer}",
        text_content=False
    )
    return loader.load()

In [73]:
# --- Process Data ---
def process_data(docs):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    return text_splitter.split_documents(docs)

# --- Create Vector Store ---
def create_vector_store(chunks):
    embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
    return Chroma.from_documents(chunks, embeddings, persist_directory="./rag_db")

# --- RAG Query ---
def rag_query(vector_db, user_query, threshold=0.3, k=5):
    relevant_docs = vector_db.similarity_search_with_relevance_scores(user_query, k=k)
    filtered_docs = [doc for doc, score in relevant_docs if score >= threshold]
    
    if not filtered_docs:
        return "Please ask about company policies."
    
    context = "\n".join([doc.page_content for doc in filtered_docs])
    response = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[
            {"role": "system", "content": "slighty elobarate the answers."},
            {"role": "user", "content": f"Context:\n{context}\n\nQuestion: {user_query}"}
        ],
        temperature=0.3,
        max_tokens=100
    )
    return response.choices[0].message.content



In [21]:
!pip install jq

In [63]:
docs = load_json("C:/Users/SGokul/OneDrive - Mastech Digital/Documents/MIT NOVA/company_policies.json")
chunks = process_data(docs)
vector_db = create_vector_store(chunks)

In [76]:
# --- Execute Pipeline ---
if __name__ == "__main__":
    print(rag_query(vector_db, "what is the purpose of webclock"))

The purpose of Webclock is to enable employees who work from home to log their attendance accurately and efficiently. This system allows remote workers to record their hours worked, ensuring that their attendance is properly tracked and recorded. By using Webclock, employees can easily log their attendance, which helps to maintain a clear and transparent record of their work hours, making it easier for employers to manage and monitor their remote workforce.


In [58]:
'''# After creating vector_db, test directly:
test_query = "notice period"
docs = vector_db.similarity_search(test_query, k=1)
print("Top match:", docs[0].page_content if docs else "No matches")'''

'# After creating vector_db, test directly:\ntest_query = "notice period"\ndocs = vector_db.similarity_search(test_query, k=1)\nprint("Top match:", docs[0].page_content if docs else "No matches")'